# Thermal Radiative Model Interface

This notebook will produce relevant figures from the accompanying publication. The first section uses a data file of measured secondary eclipses. The remaining sections are specific to planets with sampled light curves. It imports existing MCMC results for likelihood optimization of the thermal model parameters.

***
##### Import shared packages.

In [ ]:
import datetime
from glob import glob
from importlib import import_module

import astropy.constants as C
import astropy.units as U
from astropy.analytic_functions import blackbody_lambda
import numpy as N
from scipy.stats import lognorm

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import ticker
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
from colorcet import cm

##### Import instrumental response data.

In [ ]:
from data.bandpass.spitzer_IRAC import spitzer_IRAC as spitzer_IRAC
from data.bandpass.JHKs.TwoMASS import TwoMASS as TwoMASS
from data.bandpass.kepler import kepler as kepler
bandpasses = {**kepler.bandpass, **TwoMASS.bandpass, **spitzer_IRAC.bandpass}

***
## Secondary Eclipse Plots

##### Import data on measured eclipse depths in a variety of photometric bands, along with some system properties. We want to calculate the ratio of the measured depths to what the depth should be if the planet were radiating uniformly at its theoretical equilibrium temperature.

In [ ]:
eclipse_file = N.genfromtxt('files/sec_eclipses.csv', delimiter=',', names=True, usecols=(range(1,15)), comments='#')
eclipse_names = N.genfromtxt('files/sec_eclipses.csv', delimiter=',', usecols=0, dtype=str, comments='#')[1:]

In [ ]:
Teq = eclipse_file['TEFF'] * N.sqrt(0.5/eclipse_file['AR']) * U.K
Rs = eclipse_file['RSTAR'] * C.R_sun
Rp = eclipse_file['R'] * C.R_jup

In [ ]:
flux_ratio = {}
for band in bandpasses:
    I_eq = N.array([N.dot(blackbody_lambda(in_x = bandpasses[band]['wavelength']*U.um, temperature = T), bandpasses[band]['weighted_spectrum']) for T in Teq])
    I_star = N.array([N.dot(blackbody_lambda(in_x = bandpasses[band]['wavelength']*U.um, temperature = T), bandpasses[band]['weighted_spectrum']) for T in eclipse_file['TEFF']*U.K])
    F_th = I_eq / I_star * (Rp/Rs)**2
    flux_ratio[band] = eclipse_file[band] / F_th

##### Plot a scatterplot of the ratio of observed eclipse depths to the thermal depths, as a function of equilibrium temperature.

In [ ]:
plot_colors = ['#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#fee090', '#fdae61', '#f46d43', '#d73027']
plot_labels = [r'Kepler', r'$J$', r'$H$', r'$K_s$', r'3.6 $\mu$m', r'4.5 $\mu$m', r'5.8 $\mu$m', r'8.0 $\mu$m']

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

for i, band in enumerate(bandpasses):
    ax.scatter(Teq, flux_ratio[band], c=plot_colors[i], label=plot_labels[i])
    
ax.axhline(1, c='k', linewidth=0.5, linestyle='dotted', alpha=0.5)

plt.xlabel(r'T$_\mathrm{eq}$ (K)', fontsize=18)
plt.ylabel(r'F$_\mathrm{obs}$/F$_\mathrm{eq}$', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_yscale('log')
ax.set_ylim([0.5, 350])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))

plt.legend(loc='upper right', fontsize=9, ncol=2)

plt.savefig('eclipse_scatterplot.pdf', bbox_inches='tight')

##### Alternatively, we also may plot the eclipse flux ratios by rank order of temperature on the x-axis.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

x_range = N.arange(1, len(Teq)+1)
ranked_Teq = N.argsort(Teq)
bgT_array = plt.get_cmap('cet_diverging_isoluminant_cjo_70_c25')((Teq[ranked_Teq] - N.min(Teq))/N.ptp(Teq))

for i, band in enumerate(bandpasses):
    ax.scatter(x_range, flux_ratio[band][ranked_Teq], c=plot_colors[i], edgecolors='#444444', s=80, alpha=0.7, label=plot_labels[i], zorder=10)

for x in x_range:
    ax.axvspan(x-0.5, x+0.5, facecolor=bgT_array[x-1], alpha=0.25, zorder=1)

ax.axhline(1, c='k', linewidth=0.5, linestyle='dotted', alpha=0.5)

ax.set_xlabel(r'Rank by T$_\mathrm{eq}$', fontsize=24)
ax.set_ylabel(r'F$_\mathrm{obs}$/F$_\mathrm{eq}$', fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))

ax.legend(loc=(0.68,0.8), fontsize=10, ncol=2)

axT = ax.twinx()
teq_color = '#609732'
axT.step(N.r_[x_range, N.shape(Teq)[0]+1]-0.5,
         N.r_[Teq[ranked_Teq].value, N.max(Teq).value],
         where='post',
         c=teq_color, alpha=0.5, zorder=2)
axT.set_ylabel(r'T$_\mathrm{eq}$ (K)', color=teq_color, fontsize=24)
axT.tick_params('y', colors=teq_color, labelsize=16)

ax.set_xlim([x_range[0]-0.5, x_range[-1]+0.5])

plt.savefig('eclipse_scatterplot_ranked.pdf', bbox_inches='tight')

##### Plot a histogram of the eclipse ratios, including a best-fit log-normal distribution.

In [ ]:
hist_arrays = {}
for band in bandpasses:
    #Plotting cut for the histogram above a ratio of 3.
    hist_arrays[band] = (flux_ratio[band])[flux_ratio[band] < 3.]
bins = N.histogram(N.hstack(hist_arrays.values()), bins=20)[1]
lognorm_fit = lognorm.fit(N.hstack(hist_arrays.values()), loc=N.log(1.4))

plt.hist(hist_arrays.values(),
         bins=bins, color=plot_colors,
         label=plot_labels,
         stacked=True, normed=True)

x = N.linspace(0.5, 3, 100)
plt.plot(x, lognorm.pdf(x, *lognorm_fit), c='#444444', linewidth=3, linestyle='dotted', alpha=0.5)

plt.text(0.95, 0.5, '$x_0=%.3f$\n$\mu=%.3f$\n$\sigma=%.3f$'%((lognorm_fit[1], N.log(lognorm_fit[2]), lognorm_fit[0])),
         transform=plt.gca().transAxes, fontsize=12, horizontalalignment='right', verticalalignment='center')
plt.legend(loc='upper right', fontsize=10, ncol=2)
plt.xlabel(r'F$_\mathrm{obs}$/F$_\mathrm{eq}$', fontsize=16)
plt.ylabel(r'Normalized Frequency', fontsize=16)

plt.savefig('eclipse_histogram.pdf', bbox_inches='tight')

##### We can construct the equivalent of a color-magnitude diagram for eclipse depths, where magnitudes are logarithms of eclipse ratios.

In [ ]:
def CMD(bands):
    x = N.log(flux_ratio[bands[1]][N.isfinite(flux_ratio[bands[0]]) & N.isfinite(flux_ratio[bands[1]])])
    y = (N.log(flux_ratio[bands[0]])-N.log(flux_ratio[bands[1]]))[N.isfinite(flux_ratio[bands[0]]) & N.isfinite(flux_ratio[bands[1]])]
    return {'mag': x, 'color': y}

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
CMDs = [CMD(['3p6', '4p5']), CMD(['4p5', '5p8']), CMD(['5p8', '8p0'])]
plot_colors = ['#E99425', '#E96225', '#D6224C']
plot_bands = [['3.6', '4.5'], ['4.5', '5.8'], ['5.8', '8.0']]

for i, (ax, cmag) in enumerate(zip(axes, CMDs)):
    ax.scatter(cmag['mag'], cmag['color'], c=plot_colors[i])
    ax.set_xlabel(r'$\log \left(F_\mathrm{{obs}}/F_\mathrm{{eq}}\right)$, {0}$\mu$m'.format(plot_bands[i][1]),fontsize=16)
    ax.set_ylabel(r'{0}$\mu$m - {1}$\mu$m'.format(*plot_bands[i]), fontsize=16)
    
plt.tight_layout()
plt.savefig('eclipse_CMD.pdf')

***
## Planet-specific Modules

##### The planet class contains all the methods used for the geometry of the planet-star system.

In [ ]:
from planet_class import Planet

##### Import the system properties and data of the planets (contained in Python dictionarys in a separate file), into a dictionary of planet classes. For some plots we just want to pick out a specific planet, which we specify here.

In [ ]:
paths = [s.split('/')[-1] for s in glob('data/planet/*') if '__' not in s]
planets = {}
for path in paths:
    planets[path] = Planet(import_module('data.planet.{0}.{0}'.format(path)))

specific_name = 'WASP43b'

##### Import the module for the thermal model.

In [ ]:
from thermal import blackbody as thermal_model

##### Import the routine to convert surface temperatures to observed planet-star flux ratios in a given band.

In [ ]:
from data.bandpass.response import light_curve

##### Import the likelihood calculation routine.

In [ ]:
from stats.gaussian import log_likelihood
from stats.metropolis import MCMC

##### Import any visualization routines needed.

In [ ]:
from visualization import multi_animation
from visualization.style import colors
from visualization.lightcurve_plot import LightCurvePlot
from visualization.orbit_plot import OrbitPlot
from visualization.surface_plot import SurfacePlot

***
## Best-Fit Model Light Curves from Existing MCMC Results

##### A wrapper function using the thermal model and light curve routines to start with a set of specified parameter values and return a light curve. This function will be passed to the statistical likelihood function.

In [ ]:
def generate_model(planet, parameters, spectral_array):
    temperature_map = thermal_model.temperatures(planet=planet, parameters=parameters)
    model = light_curve(planet=planet,
                        temp_map=temperature_map,
                        spectral_array=spectral_array,
                        parameters=parameters,
                        use_tidal_distortion=False)
    return {'temp': temperature_map, 'model': model}

##### If there are saved MCMC outputs we want to use, import them here. Usually we have one run which converges (hopefully) to the optimum point in parameter space, as well as a run that starts at that optimum and freely explores the region around the optimum to get an estimate on the uncertainty.

##### For the MCMC routine we feed the parameters in as a list. The parameter order is:
1. Rotation period
2. Radiative timescale
3. Minimum temperature
4. Albedo

In [ ]:
mcmc_file = {}
mcmc_uncertainty_file = {}
mcmc_fit = {}
mcmc_uncertainty = {}
sigma_upper = {}
sigma_lower = {}

for name, planet in sorted(planets.items()):
    try:
        mcmc_file[name] = N.load('files/{0}_mcmc.npy'.format(planet.name.replace(" ", "")), encoding='latin1', fix_imports=True)[()]
        mcmc_uncertainty_file[name] = N.load('files/{0}_mcmc_uncertainties.npy'.format(planet.name.replace(" ", "")), encoding='latin1', fix_imports=True)[()]
    except:
        continue
    
    print(name)
    
    mcmc_fit[name] = {}
    mcmc_uncertainty[name] = {}
    sigma_upper[name] = {}
    sigma_lower[name] = {}
    
    for band in planet.data:
        mcmc_fit[name][band] = mcmc_file[name][band]['pos'][-1].squeeze() * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        print('Band: {0} um'.format(band.replace('p', '.')))
        print('Best-fit parameters:\n Rotation period = {0:.2f} ({4:.2f} PSR)\n Radiative timescale @ {5:.0f} = {1:.1f}\n Minimum temperature = {2:.0f}\n Albedo = {3:.2f}\n'.format(*mcmc_fit[name][band], mcmc_fit[name][band][0]/planet.pseudosynchronous_period(), planet.orbital_equilibrium_temperature().decompose()))
        mcmc_uncertainty[name][band] = mcmc_uncertainty_file[name][band]['pos'].squeeze()
    
        #We take the explored parameter space from the uncertainty run and generate 1-sigma bounds around the best-fit values.
        param_upper = []
        param_lower = []
        for i, fit in enumerate(mcmc_uncertainty[name][band][0]):
            upper_entries = (mcmc_uncertainty[name][band].T[i])[mcmc_uncertainty[name][band].T[i] > fit]
            if upper_entries.size == 0:
                param_upper.append(N.array([N.nan]))
            else:
                param_upper.append(upper_entries)
        for i, fit in enumerate(mcmc_uncertainty[name][band][0]):
            lower_entries = (mcmc_uncertainty[name][band].T[i])[mcmc_uncertainty[name][band].T[i] < fit]
            if lower_entries.size == 0:
                param_lower.append(N.array([N.nan]))
            else:
                param_lower.append(lower_entries)
            
        sigma_upper[name][band] = N.array([N.nanpercentile(p, 68) - mcmc_uncertainty[name][band][0][i] for i, p in enumerate(param_upper)]) * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        sigma_lower[name][band] = N.array([N.nanpercentile(p, 100-68) - mcmc_uncertainty[name][band][0][i] for i, p in enumerate(param_lower)]) * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        print('1-sigma bounds:\n Rotation period = +{0:.2f} ({4:.2f} PSR), {5:.2f} ({9:.2f} PSR)\n Radiative timescale @ {10:.0f} = +{1:.1f}, {6:.1f}\n Minimum temperature = +{2:.0f}, {7:.0f}\n Albedo = +{3:.2f}, {8:.2f}\n'.format(*sigma_upper[name][band], sigma_upper[name][band][0]/planet.pseudosynchronous_period(), *sigma_lower[name][band], sigma_lower[name][band][0]/planet.pseudosynchronous_period(), planet.orbital_equilibrium_temperature().decompose()))

##### Here we use the entire dictionary of planets to create a text file with the formatted table information for the publication.

In [ ]:
table_name = 'parameter_results.txt'
    
with open(table_name, 'w+') as table_file:
    for name, planet in sorted(planets.items()):
        if len(planet.data) > 1:
            row = '\multirow{{{0}}}{{*}}{{{1}}}'.format(len(planet.data), planet.name)
        else:
            row = planet.name
        #We test whether the planet has associated MCMC files. Otherwise, just don't include the results in the table.
        try:
            for band in planet.data:
                row_band = ' & ' + band.replace('p', '.') + ' & '
                row_band += '${0:.2f}^{{+{1:.2f}}}_{{{2:.2f}}}$ & '.format(mcmc_fit[name][band][0]/planet.pseudosynchronous_period(),
                                                                    sigma_upper[name][band][0]/planet.pseudosynchronous_period(),
                                                                    sigma_lower[name][band][0]/planet.pseudosynchronous_period())
                if round(mcmc_fit[name][band][3].value, 2) > 0.00:
                    row_band += '${0:.2f}^{{+{4:.2f}}}_{{{8:.2f}}}$ & ${1:.1f}^{{+{5:.1f}}}_{{{9:.1f}}}$ & ${2:.0f}^{{+{6:.0f}}}_{{{10:.0f}}}$ & ${3:.2f}^{{+{7:.2f}}}_{{{11:.2f}}}$'\
                                .format(*[x.value for x in mcmc_fit[name][band]], *[x.value for x in sigma_upper[name][band]], *[x.value for x in sigma_lower[name][band]])
                else:
                    row_band += '${0:.2f}^{{+{3:.2f}}}_{{{7:.2f}}}$ & ${1:.1f}^{{+{4:.1f}}}_{{{8:.1f}}}$ & ${2:.0f}^{{+{5:.0f}}}_{{{9:.0f}}}$ & $<{6:.2f}$'\
                                .format(*[x.value for x in mcmc_fit[name][band]][:-1], *[x.value for x in sigma_upper[name][band]], *[x.value for x in sigma_lower[name][band]][:-1])
                row_band += r' \\'
                row += row_band
        except:
            continue
        row = row.replace('nan', 'N/A')
        row += '\n' + r'\hline' + '\n'
        table_file.write(row)

##### For a specific planet: quick plots showing the trajectory of the MCMC walker(s) in each of the parameter dimensions, as well as the evolution of the likelihood.

In [ ]:
fig, axes = plt.subplots(5, 2, sharex=True, figsize=(12, 12))
name = specific_name
for i in range(4):
    for band in planets[name].data:
        axes[i][0].plot((mcmc_file[name][band]['pos'].T)[i][0], color=colors.color_datlab[band])
        axes[4][0].plot(mcmc_file[name][band]['logl'], color=colors.color_datlab[band])
        axes[0][0].set_title(r'Convergence Run', fontsize=20)
        
        axes[0][0].set_ylabel(r'$P_{\mathrm{rot}}/P_{\mathrm{PSR}}$', fontsize=16)
        axes[1][0].set_ylabel(r'$\tau_{\mathrm{rad}}$ (hr)', fontsize=16)
        axes[2][0].set_ylabel(r'$T_{0}$ (K)', fontsize=16)
        axes[3][0].set_ylabel(r'Albedo', fontsize=16)
        axes[4][0].set_ylabel(r'Log Likelihood', fontsize=14)
        
        axes[i][1].plot((mcmc_uncertainty_file[name][band]['pos'].T)[i][0], color=colors.color_datlab[band])
        axes[4][1].plot(mcmc_uncertainty_file[name][band]['logl'], color=colors.color_datlab[band])
        axes[0][1].set_title(r'Uncertainty Run', fontsize=20)

##### Pull the most favorable parameter values and make a grid of that single set to plot the flux curve(s). Then generate models for the most favorable parameters in each band.

In [ ]:
best_parameters = {}
best_rotation = {}
best_paramgrid = {}
best_model = {}

name = specific_name
for band in planets[name].data:
    best_parameters[band] = mcmc_fit[name][band]
    best_rotation[band] = best_parameters[band][0]
    best_paramgrid[band] = N.meshgrid(*best_parameters[band])
    
    #A time resolution of 200 is used for plotting for all planets except HD 80606 b, which uses 5000.
    planets[name].set_resolution(longitude_resolution = 72,
                                   latitude_resolution = 36,
                                   time_resolution = 200,
                                   num_orbits = 5)
    
    best_model[band] = generate_model(planets[name], best_paramgrid[band], spitzer_IRAC.bandpass[band])

##### For our transit depth analysis, we can optionally modify the data by translating the in-transit data up by the geometric transit depth. Then, use delta_data instead of planet.data.

In [ ]:
delta_data = {}
for band in planets[name].data:
    delta_data[band] = {'t':planets[name].data[band]['t'],
                        'flux': None,
                        'occultation':planets[name].data[band]['occultation']}

transit_dip = (planets[name].rp/planets[name].R)**2
print('Geometric transit depth: {0}'.format(transit_dip))

for band in delta_data:
    delta_data[band]['flux'] = N.where(planets[name].data[band]['occultation']==b't',
                                       planets[name].data[band]['flux'] + transit_dip,
                                       planets[name].data[band]['flux'])

##### Initialize the light curve with our best-fit models in each model, then draw with arbitrary smoothing for the shaded region representing rough 1- and 2-sigma boundaries on the model light curves.

In [ ]:
uncertainty_models = N.load('files/uncertainty_lightcurves.npy', encoding='latin1', fix_imports=True)[()]

In [ ]:
sigma_curves = {'resolution': 200}
for band in planets[name].data:
    models = N.array(uncertainty_models[name][band]['model'])
    counts = uncertainty_models[name][band]['count']
    flux_step = N.squeeze(N.repeat(models, counts, axis=0))
    sigma_curves[band] = {'lower': [], 'upper': []}
    for sigma in [68, 95]:    
        lower_flux = N.nanpercentile(flux_step, 50-sigma/2, axis=0)
        sigma_curves[band]['lower'].append(lower_flux)
        upper_flux = N.nanpercentile(flux_step, 50+sigma/2, axis=0)
        sigma_curves[band]['upper'].append(upper_flux)

In [ ]:
lightcurve_plot = LightCurvePlot(planet=planets[name],
                                 data=planets[name].data,
                                 model=best_model,
                                 sigma_bounds = sigma_curves,
                                 parameters=best_parameters)

In [ ]:
fig, ax = plt.subplots()
lightcurve_plot.draw(axis=ax,
                     phase_overlap=0.25,
                     filetypes=['pdf'],
                     combo=False,
                     save=True)

***
## Orbit and Surface Plots

##### Draw a top-down orbit plot with equally time-spaced points.

In [ ]:
name = specific_name
#A time resolution of 32 is used for all planets except HD 80606 b, which uses 500.
orbit_plot = OrbitPlot(planets[name], time_resolution = 32)

In [ ]:
fig, ax = plt.subplots()
orbit_plot.draw_static(axis=ax, filetypes=['pdf'], save=True)

##### Draw a projection of the planet surface with a quantity (e.g. temperature) plotted as a color map. Since we're going to animate this with the orbit plot, which has a time resolution different from the generated model time resolution, we need to select out every nth timestep, n given by the ratio.

In [ ]:
selected_band = '4p5'
surface_plot = SurfacePlot(planets[name],
                           rotation_period = best_rotation[selected_band],
                           map_array = (best_model[selected_band]['temp'].reshape(planets[name].time_resolution*(planets[name].num_orbits),(planets[name].latitude_resolution)+1,(planets[name].longitude_resolution)+1)),
                           time_resolution = 32,
                           start_orbit = 3)

##### The multi_animation routine takes multiple visualizations and animates them together over a time series.

In [ ]:
orbit_plot.set_rotation_period(planets[name].P)
orbit_plot.set_num_orbits(3)
animation = multi_animation.draw(planets[name],
                                 plot_objects = [orbit_plot, surface_plot],
                                 num_orbits = 3)

***
## Miscellaneous Plots

##### One interesting plot is to look at a map of likelihoods in a 2D slice of parameter space. Here, we make a plot of rotation period versus radiative timescale, using an existing grid search with the other 2 parameters (minimum temperature, albedo) at values fixed to their best fits from MCMC. This shows us how degenerate the selection of the 2 timescale parameters is.

In [ ]:
name = specific_name
degeneracy_file = N.load('files/{0}_grid.npy'.format(name), encoding='latin1', fix_imports=True)[()]

In [ ]:
pars = degeneracy_file['par'][:]
pars[0] = (pars[0]/planets[name].P).decompose()

degeneracy_grid = degeneracy_file['logl']['4p5'].squeeze()
best_index = N.unravel_index(N.argmin(degeneracy_grid), (len(pars[0]), len(pars[1])))
logl_min = N.nanmin(degeneracy_grid)

x, y = N.mgrid[pars[0][0]:pars[0][-1]:len(pars[0])*1j, pars[1][0]:pars[1][-1]:len(pars[1])*1j]
plt.pcolor(x, y, degeneracy_grid/logl_min, cmap='cet_fire_r', vmin=1, vmax=1.5)
cb = plt.colorbar()
cb.set_label('Scaled Log Likelihood', fontsize=16)
plt.xlabel(r'$P_{\mathrm{rot}}/P_{\mathrm{orb}}$', fontsize=16)
plt.ylabel(r'$\tau_{\mathrm{rad}}$ (hr)', fontsize=16)
plt.savefig('{0}_degeneracy.pdf'.format(name), bbox_inches='tight', transparent=True)